TODO:
- many qbits in circuit after target
- removing qArith for qVarArith after testing
- ensure we have the right answer at the end
- how is best to do multicontrol not (MCX). Should we do all in one MCX operation or separate it out? Which is less noisy?
- recycling the carry bit (adder paper). Could we use the same carry bit for all additions in the circuit? Uncompute carry after each one. Noisiness vs bit-saving.


In [1]:
from qiskit import *
from qiskit import QuantumRegister
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, IBMQ
from qiskit import providers
from qiskit.providers.aer import AerSimulator
from qiskit.tools.monitor import job_monitor

import matplotlib as mpl
import random
import pylatexenc
import numpy as np
import pandas as pd
from pandas import *
import copy
import os



In [2]:
#IBMQ.save_account("83eab2c789e4b9b5030af0f6cdcf31f3efe4f59d505bff881e43504814b25b12526b1aa8c2b5826af475da8c49d3ca4f99189e59287d8040cb3750b728aa6b76")
IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q")

/var/folders/x8/1hjpfzk52qg5mqw0049w64xm0000gp/T/ipykernel_86922/113811590.py:2: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.load_account()
/var/folders/x8/1hjpfzk52qg5mqw0049w64xm0000gp/T/ipykernel_86922/113811590.py:2: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account()


# Quantum Circuit Class

In [3]:
##
##  A class to wrap the qiskit QuantumCircuit
##    It can allocate registers in sequence, which is very useful
##
class QC:
    def __init__(self, genBarriers=True):
        self.regNum = 0
        self.genBarriers = genBarriers
        self.qcx = QuantumCircuit()
        self.falseBit = self.addReg('false')
        self.trueBit  = self.addReg('true')
        self.qcx.x(self.trueBit)
        
    def getQuantumCircuit(self):
        return self.qcx

    def get_name(self, prefix=""):
        name = prefix+"_"+str(self.regNum)
        self.regNum+=1
        return name        
    
    def addReg(self, prefix="t"):
        name = self.get_name(prefix)
        result = QuantumRegister(1, name)
        self.qcx.add_register(result)
        return result

##
##  Measure the specified qubit onto a new classical bit
##     returns:  the classical register
##
    def mbit(self, qbit):
        result = ClassicalRegister(1, "m"+qbit.name)
        self.qcx.add_register(result)
        self.qcx.measure(qbit, result)
        return result

##
##  Put in barriers, or not, as dicitated by self.genBarriers
##
    def barrier(self):
        if self.genBarriers:
            self.qcx.barrier()
##
##  Emulate locally
##
    def em(self, shots=512):
        # em = Aer.get_backend("qasm_simulator")
        em = Aer.get_backend('aer_simulator_matrix_product_state')
        # job = execute(self.qcx, em, shots=shots)
        job = execute(self.qcx, em, shots=shots)
        hist = job.result().get_counts()
        return hist
    
##
## ToDo:  pick the method according to the number of qubits
## NB:   the usual simulator was ibmq_qasm_simulator by default
##    but we're using the one better suited to our purposes
##    Possiblities for method:
##        ibmq_qasm_simulator
##        simulator_mps
##
##
    def bem(self, shots=512, method='simulator_mps'):
        provider = IBMQ.get_provider(hub='ibm-q')
        real_device = provider.get_backend("simulator_mps")
        print('Status:')
        print('  Operational: ', real_device.status().operational)
        print('  Pending jobs:', real_device.status().pending_jobs)
        print('  Status message:', real_device.status().status_msg)
        job = execute(self.qcx, real_device, shots=shots)
        job_monitor(job, interval=2)
        res = job.result()
        return res.get_counts()
    
    def sv(self):
        svs = BasicAer.get_backend("statevector_simulator")
        job = execute(self.qcx, svs)
        ket = job.result().get_statevector()
        return ket
    
    
##
## Idea: split from here below into another decorator class
##
    
    def genOutput(self, reg, name):
        reg = reg if reg else self.addReg(name)
        return reg
    
    def qand(self, bit1, bit2, output=None):
        output = self.genOutput(output, "and")
        self.barrier()
        self.qcx.ccx(bit1, bit2, output)
        self.barrier()
        return output


    def qor(self, bit1, bit2, output=None):
        output = self.genOutput(output, "or")
        self.barrier()
        self.qcx.cx(bit1, output)
        self.qcx.cx(bit2, output)
        self.qcx.ccx(bit1, bit2, output)
        self.barrier()
        return output
    
    def qxor(self, bit1, bit2, output=None):
        output = self.genOutput(output, "xor")
        self.barrier()
        self.qcx.cx(bit1, output)
        self.qcx.cx(bit2, output)
        self.barrier()
        return output
##
##  While we don't need this method for completeness,
##     it can perform a 3-way xor without generating a
##     temporary, intermediate result, thus saving qubits
##
    def qxor3(self, bit1, bit2, bit3, name="xor3", output=None):
        output = self.genOutput(output, name)
        self.barrier()
        self.qcx.cx(bit1, output)
        self.qcx.cx(bit2, output)
        self.qcx.cx(bit3, output)
        self.barrier()
        return output

    def qnot(self, bit, output=None):
        output = self.genOutput(output, "not")
        self.barrier()
        self.qcx.x(bit)
        self.qcx.cx(bit, output)
        self.qcx.x(bit)
        self.barrier()
        return output

    def qnand(self, bit1, bit2, output=None):
        output = self.genOutput(output, "nand")
        self.barrier()
        self.qcx.ccx(bit1, bit2, output)
        self.qcx.x(output)
        self.barrier()
        return output

###
### A more qulever carry computation, requiring no intermediate results
###   If exactly two bits are true, the output is flipped once, making it True
##    If all three are true, the output flips 3 times, making it True
##    If exactly one bit is true, no flipping occurs
##
    def quarry(self, bit1, bit2, bit3, output=None):
        output = self.genOutput(output, "quarry")
        self.barrier()
        self.qcx.ccx(bit1, bit2, output)
        self.qcx.ccx(bit2, bit3, output)
        self.qcx.ccx(bit1, bit3, output)
        self.barrier()
        return output
    
    
    def draw(self, output='mpl'):
        return self.qcx.draw(output=output)

# Quantum Arithmetic Class

In [5]:
##
##  Create integers and sums for the supplied quantum circuit
##    Throughout, a value is represented as a list of quantum registers,
##      least-significant-bit-first
##
class QVarArith:
    def __init__(self, qc):
        self.qc = qc
        self.ONE   = self.qint(1, "one")

    def numBits(self, val):
        if val == 0:
            return 1
        ans = int(np.log2(val))+1
        #print("For val " + str(val) + " " + str(ans) + " bits")
        return ans
##
##  Create the supplied value on the quantum circuit
##
    def qint(self, val, name=None, width=None):
        if not width:
            width = self.numBits(val)
        if not name:
            name = "qi("+str(val)+")"
        bitVal = []
        a = 1 #this would be the eventual separation point if we want to separate qint into defining width and values
        for i in range(width):
            t = self.qc.addReg(name)
            if val & a:
                self.qc.getQuantumCircuit().x(t)
            a = a << 1
            bitVal.append(t)
        return bitVal
    
#this is so our shadow registers for qvar can have defined length w/out defined value
    def qtemp(self,val,name=None):
        width = self.numBits(val)
        if not name:
            name = "qtempshad("+str(val)+")"
        bitVal = []
        for i in range(width):
            t = self.qc.addReg(name)
            bitVal.append(t)
        return bitVal
##
##  cin -- carry in
##  ba - one bit of input
##  bb -- other bit of input
##  returns the sum and carry bits
##  Note:  this is correct but chews up many temporary qubits
##
    def onebit(self, cin, ba, bb):
        ##
        ##  Using a 3-way :-) to save on qubits
        ##
        s = self.qc.qxor3(cin, ba, bb, name="add")
        ##
        ##  https://en.wikipedia.org/wiki/Adder_(electronics)#Full_adder
        ##  ba bb   +   cin (ba xor bb)
        # cout = self.qc.qor(self.qc.qand(ba,bb), self.qc.qand(cin,self.qc.qxor(ba,bb)))
        ##  Not using the above formuala, correct though it be
        ##    It generates 3 intermediate results, but the line below generates none
        ##
        cout = self.qc.quarry(cin, ba, bb)
        return s, cout
    
    def onebitshort(self, cin, ba, output=None):
        output = self.qc.genOutput(output, "quarry")
        s = self.qc.qxor(cin, ba)
        
        self.qc.getQuantumCircuit().ccx(cin, ba, output)
        return s, output
##
##  add the two inputs, returning the result
##
##  NB:  There is an opportunity to save one qubit here
##     The onebit call always generates both the sum s and the carry out, 
##        received as carry.  The carry is needed in the middle but not for
##        the very last onebit call
##     Making onebit aware of this would make the code ugly, I think [ RKC ]
##     Perhaps better would be to compute the sum and carry separately
##        and avoid the carry the last time. The onebit method is really just
##        those two calls, anyway, so this wouldn't be so hard.
##
    def add(self, val1, val2, width = 10):
        if len(val1) > len(val2):
            return self.add(val2, val1, width)
        ans   = []
        carry = self.qc.falseBit
        for i in range(len(val2)):
            if i<len(val1):
                s, carry = self.onebit(carry, val1[i], val2[i])
                ans.append(s)
            else:
                ##only need to add val2 and the carry at this point
                s, carry = self.onebitshort(carry, val2[i])
                ans.append(s)
        while len(ans) < width:
            ans.append(carry)
            carry = self.qc.addReg('pad')
        return ans
    
    def onescomp(self, val):
        self.qc.barrier()
        width = len(val)
        ans = []
        for i in range(width):
            ans.append(self.qc.qnot(val[i]))
        self.qc.barrier()
        return ans
            
    def neg(self, val): 
        return self.add(self.onescomp(val), self.ONE, len(val))
    
    def measure(self, val):
        width = len(val)
        result = []
        for i in range(width):
            b = self.qc.mbit(val[i])
            result.append(b)
        return result
    
    

Quantum Arithmetic Example 

In [8]:
##
## 32 bit addition problem
##
qc = QC()
qi = QVarArith(qc)
ans = qi.qint(0)
for i in range(1,10):
    n = qi.qint(i)
    ans = qi.add(ans,n)
m = qi.measure(ans)
#qc.draw()
#qc.em(shots=16)

# Subset Sum Class

In [13]:
class QSubsetSum:
    
    def __init__(self, values, target, qc=None):
        if not qc:
            self.qc = QC()
        else:
            self.qc = qc
        self.values = values
        self.target = target
        # print(self.target)
    #move tester outside class
    
    
    def run(self, sorted = True, partial = True, variable=True):
        if sorted:
            self.values.sort()
        widthOfSum = int(np.log2(sum(self.values)))+1 #placeholder: width = log2(sum) -- code this
        self.qi = QVarArith(self.qc)
        if variable:
            qintwidth = None
        else:
            qintwidth = widthOfSum
        
        self.x = []
        for i in range(len(self.values)): #creates x register
            r = self.qc.addReg(prefix = "x")
            self.x.append(r)
            ##
            ##  go ahead and call for the superposition of all
            ##   possible x values
            ##
            self.qc.getQuantumCircuit().h(r)
        print(self.x)
        ##
        ##  sorted and unsorted versions
        ##
        ##  Create the running sum of integers in my (unsorted) set
        ##  ans maintains the running sum, and is returned at thend
        ##  By "sum" we mean that each integer in the set is either added
        ##    into the sum, or not, depending on the x control register
        ##  
        ##  ans starts out as a new register, all 0
        ##
        ans = self.qi.qint(0, name = "sum")
        
        if partial:
            totsum = 0
        else:
            totsum = int(sum(self.values))
        for i, s in enumerate(self.values): #stores integers as quantum registers
            r = self.qi.qint(s)
            ctrl = self.x[i]
            if partial:
                totsum += self.values[i]
            # print(partialSum)
            ##make qtemp function that allocates result reg as big as s
            shadow = self.qi.qtemp(s, name = "shd")
            # print(len(shadow))
            for a in range(len(shadow)):
                self.qc.getQuantumCircuit().ccx(ctrl, r[a], shadow[a]) #
            ans = self.qi.add(ans, shadow, self.qi.numBits(totsum))
            # ans = self.qi.add(ans, shadow, self.bigSum)
            # print("ans", ans)
        targ = self.qi.qint(self.target, name = "target", width=len(ans))
        targ = self.qi.neg(targ)
        subtraction = self.qi.add(ans, targ, self.qi.numBits(totsum))
        self.qi.measure(subtraction)
        # self.qi.measure(ans)
        print(self.qc.regNum)
        return subtraction

    
    ##
    ## variable: use variable length addition, if false used fixed
    ## this is identical to run execept it returns the # of registers instead of the subtraction result.
    def testRun(self, sorted = True, partial = True, variable=True):
        if sorted:
            self.values.sort()
        widthOfSum = int(np.log2(sum(self.values)))+1 #placeholder: width = log2(sum) -- code this

        self.qi = QVarArith(self.qc)
        if variable:
            qintwidth = None
        else:
            qintwidth = widthOfSum
        
        self.x = []
        for i in range(len(self.values)): #creates x register
            r = self.qc.addReg(prefix = "x")
            self.x.append(r)
            self.qc.getQuantumCircuit().h(r)
        ans = self.qi.qint(0, name = "sum")
        
        if partial:
            totsum = 0
        else:
            totsum = int(sum(self.values))

        for i, s in enumerate(self.values): #stores integers as quantum registers
            r = self.qi.qint(s, width=qintwidth)
            ctrl = self.x[i]
            if partial:
                totsum += self.values[i]
            shadow = self.qi.qtemp(s, name = "shd")
            for a in range(len(shadow)):
                self.qc.getQuantumCircuit().ccx(ctrl, r[a], shadow[a]) #
            ans = self.qi.add(ans, shadow, self.qi.numBits(totsum))
            
        targ = self.qi.qint(self.target, name = "target", width=len(ans))
        targ = self.qi.neg(targ)
        subtraction = self.qi.add(ans, targ, self.qi.numBits(totsum))
        self.qi.measure(subtraction)
        return self.qc.regNum
        


# Qubit Usage Testing

In [10]:
def tester(numVals, maxVal=None):
        
        if not maxVal:
            maxVal = 64
        randomlist = []
        for i in range(int(numVals)):
            n = random.randint(1,maxVal)
            randomlist.append(n)
        target = random.randint(1,sum(randomlist))
        
        print("number list: "+str(randomlist)+ "|  target: "+str(target))
        qsub1 = QSubsetSum(copy.deepcopy(randomlist), target)
        ans = qsub1.testRun(True, True, True) #varArith with sorted values and partial sum
        qsub2 = QSubsetSum(copy.deepcopy(randomlist), target)
        ans2 = qsub2.testRun(False, True, True) #varArith with unsorted values and partial sum
        qsub3 = QSubsetSum(copy.deepcopy(randomlist), target)
        ans3 = qsub3.testRun(False, False, True) #varArith with unsorted values and total sum
        qsub4 = QSubsetSum(copy.deepcopy(randomlist), target)
        ans4 = qsub4.testRun(True, True, False) #fixedArith with sorted values and partial sum
        return ans, ans2, ans3, ans4

In [14]:
d = {'sorted,partial,varArith':[], 'unsorted,partial,varArith':[], 'unsorted,bigSum,varArith':[], 'fixedArith':[]}
df = pd.DataFrame(data=d)

#loop over all numvals of tester
for i in range(10):
    test = tester(10)
    df.loc[len(df.index)] = [test[0], test[1],test[2], test[3]]

display(df)
os.makedirs('tests', exist_ok=True)  
df.to_csv('tests/test1.csv')  


number list: [11, 40, 12, 5, 47, 31, 7, 33, 49, 51]|  target: 39
number list: [2, 28, 37, 16, 13, 46, 46, 48, 27, 27]|  target: 207
number list: [54, 50, 47, 59, 28, 54, 10, 54, 5, 6]|  target: 78
number list: [54, 60, 27, 53, 4, 10, 47, 45, 52, 53]|  target: 256
number list: [62, 18, 4, 15, 59, 35, 39, 4, 13, 43]|  target: 279
number list: [2, 1, 43, 16, 31, 34, 46, 47, 63, 58]|  target: 203
number list: [51, 39, 55, 8, 62, 33, 29, 11, 6, 33]|  target: 14
number list: [47, 42, 32, 15, 26, 22, 27, 42, 6, 44]|  target: 106
number list: [16, 12, 2, 34, 6, 42, 58, 55, 50, 47]|  target: 297
number list: [17, 31, 7, 44, 10, 62, 33, 52, 22, 4]|  target: 83


,"sorted,partial,varArith","unsorted,partial,varArith","unsorted,bigSum,varArith",fixedArith
0,290,305,341,331
1,292,305,341,332
2,298,331,347,337
3,314,333,353,350
4,288,315,343,329
5,286,291,339,327
6,299,331,349,337
7,307,325,349,345
8,292,305,344,332
9,286,310,342,327


# Subtraction Result Testing

In [ ]:
s = [2,9,26,7,12]
# s = [2,9] 
### pre target
# [2,7] -> partialSum - 26 qbits, bigSum - 29 qbits
# [2,9,7] -> sorted partialSum - 44 qbits, unsorted partialSum - 45qbits, bigSum - 51 qbit
# [2,9,26,7,12] -> sorted partialSum - 86 qbits, unsorted partialSum - 92qbits, bigSum - 101 qbit

###post target
# [2,9,26,7,12] -> sorted partialSum - 122 qbits, unsorted partialSum - 92qbits, bigSum - 101 qbit

target = 0
qsub = QSubsetSum(s, target)

# qc, test = tester(2)
# print(test)
qsub.run()
# too big for now
# qc.draw('mpl')

def twosToBinary(binary_str):
    binary_str = binary_str.replace(" ", "")
    is_negative = binary_str[0] == '1'
    if is_negative:
            inverted_bits = ''.join('1' if bit == '0' else '0' for bit in binary_str[1:])
            binary_str = bin(int(inverted_bits, 2) + 1)[2:]
    decimal_value = int(binary_str, 2)
    if is_negative:
        decimal_value = -decimal_value
    return decimal_value

# print(res.keys())
res = qc.em()
newdict = {}
newdict['Value'] = 'Counts'
for key in res.keys():
    newkey = twosToBinary(key)
    newdict[newkey]=[res[key]]
df = pd.DataFrame.from_dict(newdict)
df = df.transpose()
df = df.rename(columns={0: ''})
print(df)
# print(res)
